# Maximum Likelihood 

Modules

In [ ]:
#%%
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sns
import math
from sympy import *
import scipy.stats as stats
from sympy.utilities.lambdify import NUMPY_TRANSLATIONS
from scipy import optimize 

Likelihood function without using logarithm

In [ ]:
def likelihood(x,theta):
    num_heads, num_tails = x
    return stats.binom.pmf(num_tails,(num_heads+num_tails),theta) 

For ex. (4,6); what is probability of getting 6 tails in 10 times in which probability of getting tail is theta/0.9

In [ ]:
x = [ (4,6),(3,7),(5,5),(1,9),(7,3),(8,2),(6,4) ] 
n = 10

likelihood(x[1],0.9)

0.05739562799999997

Likelihood function by using logarithm

In [ ]:
def log_likelihood(x,theta):
    num_heads, num_tails = x
    return stats.binom.logpmf(num_tails,(num_heads+num_tails),theta)
    # to no deal small #s, we are using log
    
log_likelihood(x[1],0.9)

-2.857787145804875

Using `scipy.optimize.minimize` to determine probability

In [ ]:
def neglogll(theta,x):
    num_heads, num_tails = x
    return -1 * stats.binom.logpmf(num_tails,(num_heads+num_tails),theta)

optimize.minimize(neglogll, [0.6], args=[3,6], bounds = [(0,1)],method= 'tnc') 

     fun: array([1.29781072])
     jac: array([-1.22568621e-05])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 18
     nit: 4
  status: 1
 success: True
       x: array([0.66666636])

---
---
## Maximum Likelihood Experiments

We define coin experiments by 10 flips and recorded how many times tails and heads occur

Quest: `Can we find probabilty of occurance of tails in respect to specific coin?`

Let's say we have two types of coin: `0` and `1`

In [ ]:
xs = [ (4,6),(3,7),(5,5),(1,9),(7,3),(8,2),(6,4)] 
zs = [0,1,0,1,0,0,0] 

Defining Maximum Likelihood

This function has no further parameters and there is no other function to calculate. 

In [ ]:
def mle(x):
    num_heads, num_tails = x
    return (num_tails+0.0000000000001) / ((num_heads+num_tails)+0.00000000001) 

In [ ]:
def find_thetas(xs,zs):
    coin1tails=0
    coin1heads=0
    coin2tails=0
    coin2heads=0
    for x,z in zip(xs,zs):
        num_heads,num_tails = x
        if z == 0:
            coin1tails += num_tails
            coin1heads += num_heads
        else:
            coin2tails += num_tails
            coin2heads += num_heads
    return mle((coin1heads,coin1tails)),mle((coin2heads,coin2tails)) 

We are allowed to see that coin 0 has 0.399, and coin 1 has 0.799 chance to get tail. 

In [ ]:
find_thetas(xs,zs) 

(0.39999999999992203, 0.7999999999996049)

---
Quest: `Can we find specific coins by looking to experiment and probabilities?`

In [ ]:
xs = [ (4,6),(3,7),(5,5),(1,9),(7,3),(8,2),(6,4)] 
thetas = [0.5,0.6] 

Defining Maximum Likelihood

This function has no further parameters and there is no other function to calculate. 

In [ ]:
def logll(x,p):
    num_heads,num_tails = x
    p_tails = p
    p_heads = 1-p
    return num_tails * np.log(p_tails) + num_heads * np.log(p_heads) 

In [ ]:
def find_zs(xs,thetas):
    theta_a = thetas[0]
    theta_b = thetas[1]
    zs = []
    total_logll = 0
    for x in xs:
        logll_a = logll(x,theta_a) 
        logll_b = logll(x,theta_b)
        # print(x,logll_a,logll_b)
        if logll_a > logll_b:
            zs.append(0)
            total_logll += logll_a
        else:   
            zs.append(1)
            total_logll += logll_b
    return zs, total_logll


We are able to see which experiment belongs to which coin by calculating likelihoods

In [ ]:
find_zs(xs,thetas)

([1, 1, 0, 1, 0, 0, 0], -46.294376800242844)

---
---
# Expectation Maximization 

Algorithm: 
* Finding total loggs/zs, record likelihood, reassign everything,
* keep doing this until likelihodd ddoes not improve
Protocol:

`What is best initial guess?`

* #1 start w initial guess for thetas 
* #2 use these thetas and assign to one coin and total log
* #3 using these assgments, update our parameters, find new probabilties for each coin
* #4 report what`s happened 
* #5 compare current liklelihood w to the last annd if it less than certain small value ,
* #6 break because our likelihood is not improving in that case. 
* #7 our current likelihood becomes last likelihood and we go back and do it again. 

In [ ]:
def coin_em(cs,initial_guess = [0.1,0.9]): #1
    max_iter = 100 # len(cs) makes more sense
    tol = 0.0001 #6
    thetas = initial_guess
    last_logll = -np.infty
    for i in range(max_iter):
        #E Step    
        zs,total_logll = find_zs(cs,thetas) #2
        # determine which result belongs to which type of coin
        #M Step 
        thetas = find_thetas(cs,zs) #3 
        # determine probability of each of them by looking their mle
        print(f'Iteration {i}:') #4
        print(f' Thetas = {thetas}')
        print(f'Current log likelihood = {total_logll}')
        if total_logll - last_logll < tol: #5
            break
        last_logll = total_logll
    return thetas,zs  



We can introduce experiments and provide initial guess. Then we are able to see, probabilities and coin types according to results of experiments.  

In [ ]:
coin_em([(10, 0), (5, 5), (9, 1), (4, 6),], initial_guess = [0.1,0.9] ) 

Iteration 0:
 Thetas = (0.19999999999993667, 0.5999999999994101)
Current log likelihood = -26.186666399675243
Iteration 1:
 Thetas = (0.04999999999997999, 0.5499999999997299)
Current log likelihood = -19.714863835693734
Iteration 2:
 Thetas = (0.04999999999997999, 0.5499999999997299)
Current log likelihood = -17.733081141189217
Iteration 3:
 Thetas = (0.04999999999997999, 0.5499999999997299)
Current log likelihood = -17.733081141189217


((0.04999999999997999, 0.5499999999997299), [0, 1, 0, 1])